# Zipline Coding Exercises

**Introduction**
On any given trading day, the entire universe of stocks consists of thousands of securities. Usually, you will not be interested in investing in all the stocks in the entire universe, but rather, you will likely select only a subset of these to invest. For example, you may only want to invest in stocks that have a 10 -day average closing price of $\$ 10.00$ or less. Or you may only want to invest in the top 500 securities ranked by some factor.
In order to avoid spending a lot of time doing data wrangling to select only the securities you are interested in, people often use pipelines. In general, a pipeline is a placeholder for a series of data operations used to filter and rank data according to some factor or factors.
In this notebook, you will learn how to work with the Zipline Pipeline. Zipline is an open-source algorithmic trading simulator developed by Quantopian. We will learn how to use the Zipline Pipeline to filter stock data according to factors.

**Loading Data with Zipline**
Before we build our pipeline with Zipline, we will first see how we can load the stock data we are going to use into Zipline. Zipline uses Data Bundles to make it easy to use different data sources. A data bundle is a collection of pricing data, adjustment data, and an asset database. Zipline employs data bundles to preload data used to run backtests and store data for future runs. Zipline comes with a few data bundles by default but it also has the ability to ingest new bundles. The first step to using a data bundle is to ingest the data. Zipline's ingestion process will start by downloading the data or by loading data files from your local machine. It will then pass the data to a set of writer objects that converts the original data to Zipline's internal format ( bcolz for pricing data, and sQuite for split/merger/dividend data) that has been optimized for speed. This new data is written to a standard location that Zipline can find. By default, the new data is written to a subdirectory of ZIPLINE_ROOT/data/<bundle>, where <bundle> is the name given to the bundle ingested and the subdirectory is named with the current date. This allows Zipline to look at older data and run backtests on older copies of the data. Running a backtest with an old ingestion makes it easier to reproduce backtest results later.


In [3]:
#Install Packages

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from zipline.data import bundles

os.environ['ZIPLINE_ROOT'] = os.path.join(os.getcwd(),'data','module_4_quizzes_eod')
ingest_func = bundles.csvdir.csvdir_equities(['daily'], 'USA_bundle')
bundles.register('USA_bundle', ingest_func)
print('Data Registered')

Data Registered


C:\Users\lunar\anaconda3\envs\p36env\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Overwriting bundle with name 'USA_bundle'
  if __name__ == '__main__':


## TODO: Build an Empty Pipeline with a Screen

In [ ]:
from zipline.pipeline import Pipeline
from zipline.pipeline.factors import AverageDollarVolume

# Create a screen for our Pipeline
universe = AverageDollarVolume(window_length = 60).top(500)

# Create an empty Pipeline with the given screen
pipeline = Pipeline(screen = universe)

## TODO: Create a Pipeline Engine

**Datasets and Dataloaders**
One of the features of Zipline's Pipeline is that it separates the actual source of the stock data from the abstract description of that dataset. Therefore, Zipline employs Data Sets and Loaders for those datasets. Datasets are just abstract collections of sentinel values describing the columns/types for a particular dataset. While a loader is an object which, given a request for a particular chunk of a dataset, can actually get the requested data. For example, the loader used for the USEquityPricing dataset, is the USEquityPricingLoader class. The USEquityPricingLoader class will delegate the loading of baselines and adjustments to lower-level subsystems that know how to get the pricing data in the default formats used by Zipline ( bcolz for pricing data, and sQLite for split/merger/dividend data). As we saw in the beginning of this notebook, data bundles automatically convert the stock data into bcolz and SQLite formats. It is important to note that the USEquityPricingLoader class can also be used to load daily OHLCV data from other datasets, not just from the usequityPricing dataset. Simliarly, it is also possible to write different loaders for the same dataset and use those instead of the default loader. Zipline contains lots of other loaders to allow you to load data from different datasets.

In the code below, we will use USEquityPricingLoader (BcolzDailyBarWriter, sQLiteAdjustmentWriter) to create a loader from a bcolz equity pricing directory and a SQLite adjustments path. Both the BcolzDailyBarWriter and sQLiteAdjustmentWriter determine the path of the pricing and adjustment data. Since we will be using the Quotemedia data bundle, we will use the bundle_data.equity_daily_bar_reader and the bundle_data.adjustment_reader as our BcolzDailyBarWriter and squiteAdjustmentWriter, respectively.

In [ ]:
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.classifiers import Classifier
from zipline.pipeline.engine import SimplePipelineEngine
from zipline.pipeline.loaders import USEquityPricingLoader
from zipline.utils.numpy_utils import int64_dtype
from zipline.pipeline.domain import US_EQUITIES

class PricingLoader:
    def __init__(self, bundle_data):
        self.loader = USEquityPricingLoader.without_fx(
            bundle_data.equity_daily_bar_reader,
            bundle_data.adjustment_reader
        )
    def get_loader(self, column):
        if column not in USEquityPricingLoader.columns:
            raise Exception('Column not in USEquityPricing')
        return self.loader

class Sector(Classifier):


In [14]:
symbol_sector =['Healthcare',
 'Basic Materials',
 'Industrials',
 'Technology',
 'Healthcare',
 'Consumer Cyclical',
 'Healthcare',
 'Technology',
 'None',
 'Technology',
 'Industrials',
 'Technology',
 'Utilities',
 'Financial Services',
 'Technology',
 'None',
 'Financial Services',
 'Industrials',
 'Basic Materials',
 'Healthcare',
 'Financial Services',
 'Financial Services',
 'Technology',
 'Communication Services',
 'Technology',
 'Healthcare',
 'Real Estate',
 'Consumer Cyclical',
 'Technology',
 'Healthcare',
 'Energy',
 'Basic Materials',
 'Technology',
 'Financial Services',
 'Technology',
 'Consumer Cyclical',
 'None',
 'Technology',
 'Technology',
 'Consumer Cyclical',
 'Communication Services',
 'Technology',
 'Financial Services',
 'Healthcare',
 'Consumer Cyclical',
 'Consumer Cyclical',
 'Financial Services',
 'Healthcare',
 'Consumer Cyclical',
 'Consumer Cyclical',
 'Healthcare',
 'Industrials',
 'Real Estate',
 'Basic Materials',
 'Communication Services',
 'Consumer Defensive',
 'Healthcare',
 'Communication Services',
 'Technology',
 'Technology',
 'None',
 'Consumer Cyclical',
 'Financial Services',
 'Consumer Cyclical',
 'Healthcare',
 'Healthcare',
 'Financial Services',
 'Financial Services',
 'None',
 'None',
 'Healthcare',
 'Consumer Cyclical',
 'Industrials',
 'Financial Services',
 'Consumer Defensive',
 'Industrials',
 'Industrials',
 'Industrials',
 'Real Estate',
 'Consumer Cyclical',
 'Technology',
 'Healthcare',
 'Financial Services',
 'Consumer Cyclical',
 'Communication Services',
 'Consumer Cyclical',
 'Basic Materials',
 'Healthcare',
 'Communication Services',
 'Financial Services',
 'Consumer Cyclical',
 'Industrials',
 'None',
 'Industrials',
 'Financial Services',
 'None',
 'Technology',
 'Real Estate',
 'Energy',
 'Consumer Defensive',
 'Technology',
 'Consumer Cyclical',
 'Technology',
 'Consumer Cyclical',
 'Technology',
 'Technology',
 'Industrials',
 'Industrials',
 'Energy',
 'Technology',
 'Technology',
 'Consumer Cyclical',
 'Healthcare',
 'Energy',
 'None',
 'Consumer Cyclical',
 'Industrials',
 'Communication Services',
 'None',
 'Technology',
 'Consumer Cyclical',
 'Technology',
 'Financial Services',
 'Healthcare',
 'Consumer Cyclical',
 'Healthcare',
 'None',
 'Technology',
 'Communication Services',
 'Communication Services',
 'Consumer Cyclical',
 'Consumer Cyclical',
 'Real Estate',
 'Consumer Defensive',
 'Technology',
 'Healthcare',
 'Technology',
 'None',
 'Basic Materials',
 'Consumer Cyclical',
 'Consumer Cyclical',
 'Utilities',
 'Energy',
 'Financial Services',
 'Healthcare',
 'Consumer Cyclical',
 'Basic Materials',
 'None',
 'Industrials',
 'None',
 'Industrials',
 'Energy',
 'Technology',
 'Energy',
 'Technology',
 'Energy',
 'Real Estate',
 'Industrials',
 'Consumer Cyclical',
 'None',
 'None',
 'None',
 'None',
 'Utilities',
 'Consumer Cyclical',
 'Energy',
 'Industrials',
 'Industrials',
 'Basic Materials',
 'Industrials',
 'Technology',
 'Technology',
 'Financial Services',
 'Technology',
 'Consumer Cyclical',
 'Technology',
 'Technology',
 'Consumer Cyclical',
 'Technology',
 'Financial Services',
 'None',
 'None',
 'None',
 'Healthcare',
 'Consumer Defensive',
 'None',
 'Technology',
 'Consumer Cyclical',
 'Industrials',
 'Basic Materials',
 'Communication Services',
 'Communication Services',
 'Industrials',
 'Consumer Cyclical',
 'Healthcare',
 'None',
 'Energy',
 'Financial Services',
 'Healthcare',
 'Energy',
 'Consumer Cyclical',
 'Industrials',
 'Technology',
 'Energy',
 'Technology',
 'Technology',
 'Healthcare',
 'None',
 'None',
 'Technology',
 'None',
 'Healthcare',
 'None',
 'None',
 'Basic Materials',
 'None',
 'None',
 'None',
 'Healthcare',
 'Industrials',
 'Technology',
 'Technology',
 'Healthcare',
 'Healthcare',
 'None',
 'Financial Services',
 'Industrials',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'Industrials',
 'None',
 'Healthcare',
 'Financial Services',
 'Consumer Defensive',
 'Consumer Defensive',
 'Industrials',
 'Financial Services',
 'Technology',
 'Consumer Defensive',
 'Financial Services',
 'Technology',
 'Consumer Defensive',
 'Energy',
 'None',
 'Consumer Cyclical',
 'None',
 'None',
 'Financial Services',
 'None',
 'None',
 'Consumer Cyclical',
 'Consumer Cyclical',
 'Industrials',
 'Consumer Cyclical',
 'Healthcare',
 'Communication Services',
 'Technology',
 'Consumer Cyclical',
 'Industrials',
 'Basic Materials',
 'Technology',
 'Communication Services',
 'Financial Services',
 'Consumer Cyclical',
 'Consumer Cyclical',
 'None',
 'Technology',
 'Healthcare',
 'Technology',
 'Consumer Defensive',
 'Consumer Cyclical',
 'Financial Services',
 'Consumer Cyclical',
 'Technology',
 'Financial Services',
 'Industrials',
 'Consumer Defensive',
 'Energy',
 'Technology',
 'Healthcare',
 'Energy',
 'Technology',
 'Financial Services',
 'Technology',
 'Technology',
 'Basic Materials',
 'Communication Services',
 'Consumer Cyclical',
 'Utilities',
 'Basic Materials',
 'Technology',
 'Communication Services',
 'Consumer Cyclical',
 'Consumer Cyclical',
 'Industrials',
 'Healthcare',
 'Communication Services',
 'Technology',
 'Basic Materials',
 'Healthcare',
 'Technology',
 'Technology',
 'Healthcare',
 'Financial Services',
 'Industrials',
 'None',
 'Technology',
 'Real Estate',
 'Technology',
 'Consumer Cyclical',
 'Industrials',
 'Energy',
 'Technology',
 'Technology',
 'Technology',
 'Industrials',
 'Energy',
 'Industrials',
 'Utilities',
 'Technology',
 'Consumer Cyclical',
 'Consumer Cyclical',
 'Consumer Defensive',
 'Healthcare',
 'None',
 'Financial Services',
 'Communication Services',
 'Industrials',
 'Real Estate',
 'Technology',
 'Industrials',
 'Financial Services',
 'Basic Materials',
 'Financial Services',
 'Real Estate',
 'Energy',
 'Consumer Cyclical',
 'Energy',
 'Financial Services',
 'Technology',
 'None',
 'None',
 'Technology',
 'Communication Services',
 'Energy',
 'Healthcare',
 'Technology',
 'Consumer Cyclical',
 'Consumer Cyclical',
 'None',
 'Financial Services',
 'Technology',
 'Industrials',
 'Communication Services',
 'Industrials',
 'Consumer Cyclical',
 'Technology',
 'Consumer Defensive',
 'Financial Services',
 'Healthcare',
 'Real Estate',
 'Financial Services',
 'Consumer Cyclical',
 'None',
 'Financial Services',
 'Consumer Cyclical',
 'None',
 'None',
 'Technology',
 'Technology',
 'Basic Materials',
 'Financial Services',
 'Communication Services',
 'Energy',
 'None',
 'None',
 'Communication Services',
 'Technology',
 'Technology',
 'Financial Services',
 'None',
 'None',
 'Industrials',
 'Financial Services',
 'Technology',
 'None',
 'Communication Services',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'Utilities',
 'Real Estate',
 'None',
 'Technology',
 'Financial Services',
 'Financial Services',
 'None',
 'Industrials',
 'Technology',
 'Financial Services',
 'Healthcare',
 'Consumer Defensive',
 'Consumer Cyclical',
 'Industrials',
 'Healthcare',
 'Technology',
 'Technology',
 'Technology',
 'Consumer Defensive',
 'Consumer Cyclical',
 'Healthcare',
 'Communication Services',
 'None',
 'Financial Services',
 'Financial Services',
 'Consumer Cyclical',
 'Consumer Cyclical',
 'Technology',
 'Communication Services',
 'Communication Services',
 'Communication Services',
 'None',
 'Industrials',
 'Basic Materials',
 'Technology',
 'Financial Services',
 'None',
 'Consumer Cyclical',
 'Healthcare',
 'Industrials',
 'None',
 'Industrials',
 'Financial Services',
 'None',
 'None',
 'Financial Services',
 'Basic Materials',
 'None',
 'None',
 'None',
 'None',
 'Healthcare',
 'None',
 'None',
 'None',
 'None',
 'Communication Services',
 'Real Estate',
 'None',
 'Technology',
 'None',
 'Industrials',
 'Healthcare',
 'None',
 'None',
 'None',
 'None',
 'Healthcare',
 'Technology',
 'Technology',
 'Real Estate',
 'Financial Services',
 'Consumer Cyclical',
 'Technology',
 'Financial Services',
 'Energy',
 'Consumer Defensive',
 'Technology',
 'Real Estate',
 'Consumer Cyclical',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'Technology',
 'None',
 'None',
 'None',
 'None',
 'None',
 'Energy',
 'None',
 'Consumer Cyclical',
 'Consumer Cyclical',
 'Healthcare',
 'Technology',
 'Communication Services',
 'Healthcare']
symbol = ['A',
 'AA',
 'AAL',
 'AAPL',
 'ABBV',
 'ABNB',
 'ABT',
 'ACN',
 'ACWI',
 'ADBE',
 'ADP',
 'ADSK',
 'AEP',
 'AFL',
 'AFRM',
 'AGG',
 'AIG',
 'AL',
 'ALB',
 'ALGN',
 'ALL',
 'ALLY',
 'AMAT',
 'AMC',
 'AMD',
 'AMGN',
 'AMT',
 'AMZN',
 'ANET',
 'ANTM',
 'APA',
 'APD',
 'APH',
 'APO',
 'APPS',
 'APTV',
 'ARKK',
 'ASAN',
 'ASML',
 'ATER',
 'ATVI',
 'AVGO',
 'AXP',
 'AZN',
 'AZO',
 'BABA',
 'BAC',
 'BAX',
 'BBIG',
 'BBWI',
 'BDX',
 'BE',
 'BEKE',
 'BHP',
 'BIDU',
 'BIG',
 'BIIB',
 'BILI',
 'BILL',
 'BKKT',
 'BKLN',
 'BKNG',
 'BLK',
 'BLL',
 'BMY',
 'BNTX',
 'BRK-A',
 'BRK-B',
 'BSML',
 'BSV',
 'BSX',
 'BURL',
 'BV',
 'BX',
 'BYND',
 'CAR',
 'CARR',
 'CAT',
 'CCI',
 'CCL',
 'CDNS',
 'CERN',
 'CFG',
 'CHPT',
 'CHTR',
 'CHWY',
 'CLF',
 'CLOV',
 'CMCSA',
 'CME',
 'CMG',
 'CMI',
 'CN',
 'CNI',
 'COF',
 'COG',
 'COIN',
 'CONE',
 'COP',
 'COST',
 'COUP',
 'CPNG',
 'CRM',
 'CROX',
 'CRWD',
 'CSCO',
 'CSX',
 'CTAS',
 'CTRA',
 'CTSH',
 'CTXS',
 'CVNA',
 'CVS',
 'CVX',
 'CWI',
 'CZR',
 'DAL',
 'DASH',
 'DBE',
 'DDOG',
 'DECK',
 'DELL',
 'DFS',
 'DGX',
 'DHI',
 'DHR',
 'DIA',
 'DIDI',
 'DIS',
 'DISCA',
 'DKNG',
 'DKS',
 'DLR',
 'DLTR',
 'DOCN',
 'DOCS',
 'DOCU',
 'DOG',
 'DOW',
 'DPZ',
 'DRI',
 'DUK',
 'DVN',
 'DWAC',
 'DXCM',
 'EBAY',
 'ECL',
 'EEM',
 'EFX',
 'EMB',
 'EMR',
 'ENB',
 'ENPH',
 'EOG',
 'EPAM',
 'EPD',
 'EQIX',
 'ETN',
 'ETSY',
 'EWJ',
 'EWT',
 'EWY',
 'EWZ',
 'EXC',
 'EXPE',
 'FANG',
 'FAST',
 'FCEL',
 'FCX',
 'FDX',
 'FIS',
 'FISV',
 'FITB',
 'FIVN',
 'FL',
 'FSLR',
 'FSLY',
 'FTCH',
 'FTNT',
 'FUTU',
 'FXI',
 'GDX',
 'GDXJ',
 'GILD',
 'GIS',
 'GLD',
 'GLW',
 'GME',
 'GNRC',
 'GOLD',
 'GOOG',
 'GOOGL',
 'GPN',
 'GPS',
 'GSK',
 'GUSH',
 'HAL',
 'HBAN',
 'HCA',
 'HES',
 'HLT',
 'HON',
 'HOOD',
 'HP',
 'HPQ',
 'HUBS',
 'HUM',
 'HYG',
 'IAU',
 'IBM',
 'IDU',
 'IDXX',
 'IEF',
 'IEMG',
 'IFF',
 'IGSB',
 'IJH',
 'IJR',
 'ILMN',
 'INFO',
 'INTC',
 'INTU',
 'IQV',
 'ISRG',
 'ITOT',
 'ITUB',
 'ITW',
 'IVV',
 'IWB',
 'IWD',
 'IWF',
 'IWM',
 'IWN',
 'IWO',
 'IYR',
 'JCI',
 'JETS',
 'JNJ',
 'JPM',
 'K',
 'KDP',
 'KE',
 'KEY',
 'KEYS',
 'KHC',
 'KKR',
 'KLAC',
 'KMB',
 'KMI',
 'KRE',
 'KSS',
 'KSU',
 'KWEB',
 'L',
 'LABU',
 'LB',
 'LCID',
 'LEN',
 'LHX',
 'LL',
 'LLY',
 'LOV',
 'LRCX',
 'LULU',
 'LUV',
 'LYB',
 'LYFT',
 'LYV',
 'MARA',
 'MAT',
 'MCD',
 'MCHI',
 'MCHP',
 'MCK',
 'MDB',
 'MDLZ',
 'MELI',
 'MET',
 'MGM',
 'ML',
 'MMC',
 'MMM',
 'MNST',
 'MPC',
 'MPWR',
 'MRNA',
 'MRO',
 'MRVL',
 'MSCI',
 'MSFT',
 'MSTR',
 'MT',
 'MTCH',
 'NCLH',
 'NEE',
 'NEM',
 'NET',
 'NFLX',
 'NIO',
 'NKE',
 'NOC',
 'NRC',
 'NTES',
 'NUAN',
 'NUE',
 'NVAX',
 'NVDA',
 'NXPI',
 'OCGN',
 'OCN',
 'ODFL',
 'OIH',
 'OKTA',
 'OPEN',
 'ORCL',
 'ORLY',
 'OTIS',
 'OXY',
 'PANW',
 'PATH',
 'PAYC',
 'PAYX',
 'PBR',
 'PCAR',
 'PCG',
 'PD',
 'PDD',
 'PENN',
 'PEP',
 'PFE',
 'PFF',
 'PGR',
 'PINS',
 'PL',
 'PLD',
 'PLTR',
 'PLUG',
 'PNC',
 'PPG',
 'PRU',
 'PSA',
 'PSX',
 'PTON',
 'PXD',
 'PYPL',
 'QCOM',
 'QID',
 'QLD',
 'QRVO',
 'RBLX',
 'RDS-A',
 'REGN',
 'RIOT',
 'RIVN',
 'RL',
 'RLY',
 'RM',
 'RNG',
 'ROK',
 'ROKU',
 'ROP',
 'ROST',
 'RUN',
 'SAM',
 'SAN',
 'SAVA',
 'SBAC',
 'SBNY',
 'SBUX',
 'SCHP',
 'SCHW',
 'SCI',
 'SDOW',
 'SDS',
 'SEDG',
 'SHOP',
 'SHW',
 'SIVB',
 'SKLZ',
 'SLB',
 'SLY',
 'SMH',
 'SNAP',
 'SNOW',
 'SNPS',
 'SOFI',
 'SOXL',
 'SOXX',
 'SPCE',
 'SPGI',
 'SPLK',
 'SPLV',
 'SPOT',
 'SPRT',
 'SPXL',
 'SPXS',
 'SPXU',
 'SPYG',
 'SQQQ',
 'SRE',
 'SRG',
 'SSO',
 'STNE',
 'STT',
 'SV',
 'SVXY',
 'SWK',
 'SWKS',
 'SYF',
 'SYK',
 'SYY',
 'TCOM',
 'TDG',
 'TDOC',
 'TEAM',
 'TEL',
 'TER',
 'TGT',
 'TJX',
 'TLRY',
 'TMUS',
 'TQQQ',
 'TROW',
 'TRV',
 'TSCO',
 'TSLA',
 'TTD',
 'TTWO',
 'TWLO',
 'TWTR',
 'TZA',
 'UAL',
 'UAN',
 'UBER',
 'UBS',
 'UDOW',
 'ULTA',
 'UNH',
 'UNP',
 'UPRO',
 'UPS',
 'UPST',
 'USO',
 'UVXY',
 'V',
 'VALE',
 'VCIT',
 'VCSH',
 'VEA',
 'VEDL',
 'VEEV',
 'VEU',
 'VGK',
 'VGLT',
 'VGT',
 'VIAC',
 'VICI',
 'VIH',
 'VMW',
 'VNQ',
 'VRSK',
 'VRTX',
 'VTI',
 'VUG',
 'VWO',
 'VXUS',
 'WBA',
 'WDAY',
 'WDC',
 'WELL',
 'WFC',
 'WISH',
 'WIX',
 'WLTW',
 'WMB',
 'WMT',
 'WOLF',
 'WY',
 'WYNN',
 'XBI',
 'XHB',
 'XLB',
 'XLE',
 'XLF',
 'XLI',
 'XLK',
 'XLNX',
 'XLP',
 'XLRE',
 'XLU',
 'XLY',
 'XME',
 'XOM',
 'XOP',
 'XPEV',
 'YUM',
 'ZBH',
 'ZEN',
 'ZNGA',
 'ZTS']

sector = np.array(symbol_sector)
symbol_list = np.array(symbol)

symbol_sector_matrix = np.stack((symbol_list, sector), axis=1)
np.save("D:\FBA Quant\_Assignment\FML\Week7\sector",symbol_sector_matrix)

In [21]:
import numpy as np
ss = np.load('sector.npy')
sp = pd.DataFrame(ss)
sp.loc[sp.iloc[:,1]=='None']

,0,1
8,ACWI,None
15,AGG,None
36,ARKK,None
60,BKLN,None
68,BSML,None
69,BSV,None
92,CN,None
95,COG,None
114,CWI,None
118,DBE,None


In [ ]:
from zipline.utils.calendars import get_calendar



## TODO: Create Factors and Filters For The Pipeline

## TODO: Add Factors and Filters To The Pipeline

## TODO: Run The Pipeline